In [3]:
%matplotlib inline
import h2o
import h2o.grid

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 49 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,24 days
H2O cluster name:,H2O_started_from_R_jay_lkj507
H2O cluster total nodes:,1
H2O cluster free memory:,1.425 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [5]:
data = h2o.import_file("http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip")
train, valid, test = data.split_frame([0.8,0.1],seed = 69)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
print("%d %d %d"% (train.nrows,valid.nrows, test.nrows))

35255 4272 4451


In [12]:
y = "IsArrDelayed"
ignoreFields = [
    "ArrDelay","DepDelay","CarrierDelay",
    "WeatherDelay","NASDelay","SecurityDelay",
    "LateAircraftDelay","IsDepDelayed","IsArrDelayed","ActualElapsedTime"] #But CRSElapsedTime is fine.
xAll = [i for i in train.names if i not in ignoreFields]
xLikely = ["Month","DayOfWeek","UniqueCarrier","Origin","Dest","Distance","Cancelled","Diverted"]

In [13]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [14]:
m_def = H2OGeneralizedLinearEstimator(family = "binomial")
m_def.train(xAll, y ,train,validation_frame = valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [15]:
m_def.logloss(valid=True)

0.6233229932181956

In [18]:
g = h2o.grid.H2OGridSearch(
    H2OGeneralizedLinearEstimator(family = "binomial",lambda_search = True),
    hyper_params = {"alpha":[x*0.01 for x in range(0,100)],},
    search_criteria = {"strategy":"RandomDiscrete","max_models":8,"max_runtime_secs":30}
    )
g.train(xAll, y, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [19]:
g

      alpha                                                     model_ids  \
0    [0.79]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_5   
1    [0.76]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_7   
2     [0.5]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_1   
3    [0.39]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_3   
4    [0.36]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_4   
5    [0.16]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_6   
6    [0.15]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_8   
7    [0.12]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_44_model_2   

              logloss  
0  0.5922322635457088  
1  0.5925181373661924  
2  0.5956342688595108  
3  0.5975277835129581  
4  0.5981311828643383  
5  0.6044552100697651  
6  0.6049789384982383  
7  0.6068366384328852  


In [20]:
g2 = h2o.grid.H2OGridSearch(
    H2OGeneralizedLinearEstimator(family = "binomial",lambda_search = True),
    hyper_params = {"alpha":[0, 0.2, 0.4, 0.5, 0.6, 0.8, 0.99],},
    search_criteria = {"strategy":"Cartesian"}
    )
g2.train(xAll, y, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [21]:
g2

      alpha                                                     model_ids  \
0     [0.0]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_1   
1    [0.99]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_7   
2     [0.8]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_6   
3     [0.6]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_5   
4     [0.5]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_4   
5     [0.4]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_3   
6     [0.2]  Grid_GLM_py_3_sid_b5be_model_python_1549955582989_61_model_2   

              logloss  
0  0.5868046385246171  
1  0.5902213901830258  
2  0.5921408527376583  
3  0.5942641298879805  
4  0.5956342688595108  
5  0.5973390141600553  
6   0.602665609207675  
